In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn import preprocessing 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
submit = pd.read_csv('../input/titanic/gender_submission.csv')

In [ ]:
data = pd.concat([train,test])
data

In [ ]:
sns.countplot(data['Survived'])

In [ ]:
sns.countplot(data['Pclass'], hue=data['Survived'])

In [ ]:
sns.countplot(data['Sex'], hue=data['Survived'])

In [ ]:
sns.countplot(data['Embarked'], hue=data['Survived'])

In [ ]:
data.info()

**Name**

In [ ]:
data['Title1'] = data['Name'].str.split(", ", expand=True)[1]

In [ ]:
data['Title1'].head()

In [ ]:
data['Title1'] = data['Title1'].str.split(".", expand=True)[0]

In [ ]:
data['Title1'].head()

In [ ]:
data['Title1'].unique()

In [ ]:
data['Title1'] = data['Title1'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','the Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],
         ['Miss','Mrs','Miss','Mr','Mr','Mrs','Mrs','Mr','Mr','Mr','Mr','Mr','Mr','Mrs'])

In [ ]:
data['Title1'].unique()

In [ ]:
data['Title1'].unique()

In [ ]:
data = pd.get_dummies(data, columns=['Title1'])

In [ ]:
data.info()

In [ ]:
data['Words_Count'] = data['Name'].apply(lambda x: len(x.split()))

**Sex**

In [ ]:
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1}).astype(int)

**Age**

In [ ]:
age_mean = data['Age'].mean()
age_std = data['Age'].std()
age_fill = np.random.randint(age_mean - age_std, age_mean + age_std,)

print(age_fill)

In [ ]:
data['Age'] = data['Age'].fillna(-1)

def funa(x):
  x = np.random.randint(age_mean - age_std, age_mean + age_std,)
  return x


data['Age'] = data['Age'].apply(lambda x: funa(x) if x == -1   else x)

In [ ]:
def func(x):
  if int(x) <= 16:
    x = 0
    return x
  elif int(x) > 16 and int(x) <= 32:
    x = 1
    return x
  elif int(x) > 32 and int(x) <= 48:
    x = 2
    return x
  elif int(x) > 48 and int(x) <= 64:
    x = 3
    return x
  elif int(x) > 64:
    x = 4
    return x
  # return x

data['Age'] = data['Age'].apply(lambda x: x if x < 0   else  func(x))


**SibSp + Parch**

In [ ]:
data['Family_Size'] = data['Parch'] + data['SibSp']

**Ticket**

In [ ]:
data['Ticket_type'] = data['Ticket'].apply(lambda x: x[0:3])
data['Ticket_type'] = data['Ticket_type'].astype('category')
data['Ticket_type'] = data['Ticket_type'].cat.codes 

**Fare**

In [ ]:
data['Fare'] = data['Fare'].fillna(data['Fare'].median())

In [ ]:
def funf(x):
  if int(x) <= 7.9:
    x = 0
    return x
  elif int(x) > 7.9 and int(x) <= 14.45:
    x = 1
    return x
  elif int(x) > 14.45 and int(x) <= 31.28:
    x = 2
    return x
  elif int(x) > 31.28 and int(x) <= 512.33:
    x = 3
    return x
  elif int(x) > 512.33:
    x = 4
    return x
  # return x

data['Fare'] = data['Fare'].apply(lambda x: x if x < 0   else  funf(x))

**Cabin**

In [ ]:
data['Has_Cabin'] = data['Cabin'].apply(lambda x: 0 if type(x) == float else 1 )

**Embarked**

In [ ]:
data['Embarked'] = data['Embarked'].fillna('S')

In [ ]:
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

**data processing**

In [ ]:
data.info()

In [ ]:
import seaborn as sns
plt.figure(figsize=(14,12))
dataset = data.iloc[:892]
# dataset = dataset[['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Family_Size', 'Embarked']]
sns.heatmap(dataset.corr(), annot=True)

In [ ]:
data.drop(['SibSp','Family_Size','Age','Parch','Title1_Master'], inplace= True, axis=1)

In [ ]:
dataTrain = data[pd.notnull(data['Survived'])].sort_values(by=["PassengerId"])
dataTest = data[~pd.notnull(data['Survived'])].sort_values(by=["PassengerId"])

In [ ]:
y = dataTrain['Survived']
dataTrain = dataTrain.drop('Survived',axis=1)
dataTest = dataTest.drop('Survived',axis=1)

In [ ]:
dataTrain.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'],axis=1, inplace = True)
dataTest.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'],axis=1, inplace = True)

In [ ]:
dataTrain.info()

In [ ]:
dataTest.info()

**train model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier( n_estimators = 2000,
                             min_samples_split = 3,
                             min_samples_leaf = 5,
                             oob_score = True,
                             random_state = 1,
                             n_jobs = -1)

In [ ]:
rfc.fit( dataTrain, y)

In [ ]:
from xgboost.sklearn import XGBClassifier

gbm = XGBClassifier(
    learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 1,
 #gamma=1,
 gamma=1,                        
 subsample= 1,
 colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(dataTrain, y)

In [ ]:
X_test_final = dataTest

yprob_r_test = rfc.predict_proba(X_test_final)
yprob_x_test = gbm.predict_proba(X_test_final)

In [ ]:
yprob_mean_test = (yprob_r_test + yprob_x_test)/2.0

In [ ]:
result = yprob_mean_test.argmax(axis=1)
print(result)

In [ ]:
submit['Survived'] = result

submit['Survived'] = submit['Survived'].astype(int)
submit.to_csv('submit.csv', index= False)
